In [ ]:

import re
import nltk
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

import re
import pyLDAvis.gensim
import  numpy  as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import warnings
import string
warnings.filterwarnings('ignore')

plt.style.use('ggplot')


# FUNCTIONS

# LOWER CASE DF

In [ ]:
import nltk


from nltk.tokenize import word_tokenize

def lower_case_df(text_to_lower):
    if isinstance(text_to_lower, str):
        tokens = word_tokenize(text_to_lower)
        lowercased_tokens = [token.lower() for token in tokens]
        return ' '.join(lowercased_tokens)
    else:
        return text_to_lower

# Example usage:
random_text = 'ThiS iS A tExT tO TEsT ThE fUnCTioN oF LOWerINg cAsE oF aLL LeTTErS'
lowercased_string = lower_case_df(random_text)

print(random_text)
print("--------------------------------------------------------------------")
print(lowercased_string)  


### REMOVE HTLM SIGNS

In [ ]:
def remove_htmlTags_punctuation(text):
  
    # Define the regular expression pattern for French punctuation marks
    punctuation_pattern = r'[,?!;:…()\[\]«»—/""\'\']'
    
    # Remove HTML tags
    soup = BeautifulSoup(text, "html.parser")
    text_without_tags = soup.get_text()
    
    # Use re.sub() to remove the matched punctuation marks
    cleaned_text = re.sub(punctuation_pattern, '', text_without_tags)
    
    return cleaned_text

# Example usage:
html_text = '<p>Voici un exemple de texte avec des ponctuations et des balises HTLM : «Salut !» et \'Bonjour !\'</p>'
cleaned_text = remove_htmlTags_punctuation(html_text)
print(html_text)
print("--------------------------------------------------------------------------------------------------------")
print(cleaned_text)


### LEMMATIZER

In [ ]:
def lemmatizer(text):
    # Tokenization
    tokens = nltk.word_tokenize(text.lower())

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens 
              if token not in stop_words]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    # Join the tokens back into a single string
    cleaned_text = ' '.join(tokens)
    
    return cleaned_text

# Sample text
text = "This is an example sentence that we'll use for text cleaning. It includes stopwords and different verb tenses."


cleaned_text = lemmatizer(text)

print(text)
print("------------------------------------------------------------------------------------------------------------------------")

print(cleaned_text)

# ALL PREPROCESS FUNCTIONS INTO ONE

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup
import re

def preprocessed_text(text):
    # Define the regular expression pattern for French punctuation marks
    punctuation_pattern = r'[,?!;:…()\[\]«»—/""\'\']'
    
    # Remove HTML tags
    soup = BeautifulSoup(text, "html.parser")
    text_without_tags = soup.get_text()
    
    # Use re.sub() to remove the matched punctuation marks
    cleaned_text = re.sub(punctuation_pattern, '', text_without_tags)
    
    # Tokenization
    tokens = nltk.word_tokenize(cleaned_text.lower())

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens 
              if token not in stop_words]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    # Join the tokens back into a single string
    preprocessed_text = ' '.join(tokens)
    
    return preprocessed_text

# Sample text with HTML tags and punctuation
sample_text = """
<html>
<head><title>Sample Text</title></head>
<body>
<p>This is an example of text with HTML tags, punctuation, and some stop words. It's a sample text for preprocessing.</p>
</body>
</html>
"""

# Call the preprocessed_text function to preprocess the sample text
preprocessed_sample = preprocessed_text(sample_text)

print(sample_text)
print("------------------------------------------------------------------------------------------------------------------------")
# Display the preprocessed text
print(preprocessed_sample)


### Getting a MLB Dataframe with the top X most frequents tags with atleast a 1 in the rows

In [ ]:

def filter_by_tags(df, all_tags, X):
    # Step 4: Take the first X tags from the list
    first_tags = all_tags[:X]

    # Step 5: Filter the DataFrame based on the first X tags
    filtered_df = df[df[first_tags].any(axis=1)]

    # Step 6: Drop all columns beyond the first X tags
    filtered_df = filtered_df.drop(columns=all_tags[X:])
    filtered_df = pd.DataFrame(filtered_df)
    
    return filtered_df


# Data reading / Exploration

In [ ]:

DF_OG = pd.read_csv("DF_OG.csv")



In [ ]:
DF_OG.describe()

In [ ]:
DF_OG.info()

In [ ]:
DF_OG.head(5)

In [ ]:
# Extract tags from each row using regular expression
tags_list = [re.findall(r'<(.*?)>', row) for row in DF_OG['Tags']]

combined_tags = [tag for sublist in tags_list for tag in sublist]

# Convert the list into a set to retain unique tags
unique_tags = set(combined_tags)

unique_tags_array = np.array(unique_tags)

unique_tags_array

In [ ]:
csharp_rows = DF_OG[DF_OG['Body'].str.contains('c#', case=False)]
csharp_rows

In [ ]:
test_lemmatizer = pd.DataFrame([])
test_lemmatizer['Title'] = DF_OG['Title'].apply(remove_htmlTags_punctuation)
test_lemmatizer['Body'] = DF_OG['Body'].apply(remove_htmlTags_punctuation)

In [ ]:
csharp_rows = test_lemmatizer[test_lemmatizer['Body'].str.contains('c#', case=False)]
csharp_rows

In [ ]:

test_lemmatizer['Title'] = test_lemmatizer['Title'].apply(lemmatizer)
test_lemmatizer['Body'] = test_lemmatizer['Body'].apply(lemmatizer)


In [ ]:
csharp_rows = test_lemmatizer[test_lemmatizer['Body'].str.contains('c #', case=False)]
csharp_rows

In [ ]:
sns.histplot(np.log(DF_OG['ViewCount']))

# Data Cleaning

In [ ]:
DF_OG

In [ ]:
# Apply the removing HTML signs to the 'Questions' column
DF_OG['Title'] = DF_OG['Title'].apply(preprocessed_text)
DF_OG['Body'] = DF_OG['Body'].apply(preprocessed_text)


df_MultiLabel = DF_OG[['Title' , 'Body' , 'Tags']]

df_MultiLabel['Questions'] = df_MultiLabel['Title'] + " " + df_MultiLabel['Body']
# df_MultiLabel['Questions'] = df_MultiLabel['Questions'].apply(remove_html_tags)


df_MultiLabel = df_MultiLabel.drop(['Title' , 'Body'] , axis=1)
# Swap the positions of columns Tags and Questions
df_MultiLabel['Tags'], df_MultiLabel['Questions'] = df_MultiLabel['Questions'], df_MultiLabel['Tags'].copy()
# Swap the positions and column names of columns B and C
df_MultiLabel = df_MultiLabel.rename(columns={'Tags': 'Questions', 
                                  'Questions': 'Tags'})

df_MultiLabel



In [ ]:
# Replace "c #" with "c#"
df_MultiLabel['Questions'] = df_MultiLabel['Questions'].str.replace('c #', 'c#')


In [ ]:
# Extract tags from each row using regular expression
tags_list = [re.findall(r'<(.*?)>', row) for row in df_MultiLabel['Tags']]

# Combine all tags into a single list
combined_tags = [tag for sublist in tags_list for tag in sublist]

# Convert the list into a set to retain unique tags
unique_tags = set(combined_tags)

# Count the number of unique tags
num_unique_tags = len(unique_tags)

print("Number of unique tags:", num_unique_tags)


In [ ]:
# Count the occurrences of each tag
tag_counts = {}
for tag in combined_tags:
    if tag in tag_counts:
        tag_counts[tag] += 1
    else:
        tag_counts[tag] = 1

# Sort the tags based on their counts in descending order
sorted_tags = sorted(tag_counts.items(), key=lambda x: x[1], reverse=True)

print("Repeated tags (sorted by count) FORMAT => TAG : COUNT:")
for tag, count in sorted_tags:
    print(f"Tag: {tag} : {count}")

In [ ]:
# Count the occurrences of each tag
tag_counts = pd.Series(combined_tags).value_counts()

# Sort the counts in descending order
sorted_counts = tag_counts.sort_values(ascending=False)

# Calculate the sum of the repeated numbers
sum_repeated_numbers = sorted_counts.sum()

print("Sum of repeated numbers:", sum_repeated_numbers)

# Generate normal numbers for x-axis
x_values = list(range(1, len(sorted_counts) + 1))

# Plot the graph using sns.histplot
sns.histplot(x=x_values, bins='auto', weights=sorted_counts.values, kde=False, color='b')
plt.xlabel('Number')
plt.ylabel('Repetition Count')
plt.title('Repetition Count Histogram')
plt.grid(True)
plt.show()

In [ ]:
repeated_values = pd.DataFrame(sorted_tags)[1]

# Slice the list to take the first 100 values
first_100_values = repeated_values[:100]

# Create a plot
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(first_100_values) + 1), first_100_values)

plt.xlabel('Index')
plt.ylabel('Values')
plt.title('Graph of First 100 Repeated Values')
plt.grid(True)
plt.show()

In [ ]:
df_MultiLabel['Tags'] = df_MultiLabel['Tags'].str.findall(r'<(.*?)>')

# df_MultiLabel['Questions'] = df_MultiLabel['Questions'].apply(lambda x: re.sub(r'[^\w\s]', '', x))

all_tags = pd.DataFrame(sorted_tags)[0]

df_non_MLB =  df_MultiLabel
df_non_MLB

# Multi Labeling the DataFrame

In [ ]:
MLB = MultiLabelBinarizer(classes=all_tags)
tags_binary = MLB.fit_transform(df_MultiLabel['Tags'])

# Convert the binary representation to a DataFrame
tags_df = pd.DataFrame(tags_binary, columns=MLB.classes_)

# Step 3: Concatenate the binary representation to the original DataFrame
df_MultiLabel = pd.concat([df_MultiLabel, tags_df], axis=1)
df_MultiLabel.drop('Tags' , axis=1 , inplace=True)


## Getting the value of how many tags to have 90% of the original data

In [ ]:
percentage_targeted_value = 90

filtered_rows_percent = []  # List to store filtered_df.shape[0] values in percentage

for labels_number in range(101):
    filtered_df = filter_by_tags(df_MultiLabel, all_tags, labels_number)
    percent = (filtered_df.shape[0] / DF_OG.shape[0]) * 100  # Convert to percentage
    filtered_rows_percent.append(percent)

# Find the index where the Y value is closest to 90%
index_90_percent = filtered_rows_percent.index(min(filtered_rows_percent, key=lambda x: abs(x - percentage_targeted_value)))
x_90_percent = index_90_percent  # X value where Y is closest to 90%
y_90_percent = filtered_rows_percent[index_90_percent]  # Y value closest to 90%

# Create a plot with  90% threshold
plt.figure(figsize=(10, 6))
plt.plot(range(101), filtered_rows_percent, marker='o')

plt.axhline(y=percentage_targeted_value, color='g', linestyle='--', label='90% Threshold')
plt.axvline(x=x_90_percent, color='g', linestyle='--', label=f'X at {x_90_percent}')
plt.annotate(f'({x_90_percent}, {y_90_percent:.2f}%)', (x_90_percent, y_90_percent + 2), color='g', xytext=(x_90_percent + 1, y_90_percent + 4))

plt.xlabel('Number of Tags')
plt.ylabel('Taille du data frame (%)')
plt.title('Nombre de Tags en feature par rapport a la taille original (%)')
plt.grid(True)
plt.legend()
plt.show()


In [ ]:
labels_number = 60
#Takign top 60 tags where there is atleast 1 on those tags
filtered_df = filter_by_tags(df_MultiLabel  , all_tags , labels_number)

# Sample X % of the data (you can adjust the fraction as needed)
sample_df = filtered_df.sample(frac=1, random_state=42)
df_MLB = sample_df.sort_index()

In [ ]:
filtered_df.shape[0]

# Double checking

In [ ]:

# Drop the "Questions" column
sample_df_without_questions = df_MLB.drop(columns=["Questions"])

# Sum of feature values excluding the "Questions" column
sum_of_features = sample_df_without_questions.sum()

# Check if any sum is equal to 0
any_zero_value = (sum_of_features == 0).any()

print("Sum of feature values:")
print(sum_of_features)
print("\nAre there any sums equal to 0?")
print(any_zero_value)

In [ ]:
indices_to_display = sample_df.index.tolist()  # Get the list of index values from sample_df

# Create a new DataFrame containing the rows with the specified index values from df_non_MLB
df_non_MLB = df_non_MLB.loc[indices_to_display]
df_non_MLB = df_non_MLB.sort_index()

df_OG_TT = DF_OG[['Title' , 'Body' , 'Tags']]
df_OG_TT = df_OG_TT.loc[indices_to_display]
df_OG_TT = df_OG_TT.sort_index()

In [ ]:
df_OG_TT

In [ ]:
df_non_MLB

In [ ]:
df_MLB

In [ ]:
csharp_rows = df_non_MLB[df_non_MLB['Questions'].str.contains('c#', case=False)]
csharp_rows

In [ ]:
df_OG_TT['Tags'] = df_OG_TT['Tags'].str.findall(r'<(.*?)>')
df_OG_TT

In [ ]:
df_MLB.columns

# MODELISATION

In [ ]:

from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score

import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torch
from sklearn.metrics import f1_score
from tqdm import tqdm

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import jaccard_score

from gensim.models.phrases import Phrases, Phraser
from torch.optim.lr_scheduler import ReduceLROnPlateau

import gensim


from gensim.models import CoherenceModel
from gensim import corpora, models
import pyLDAvis
import pyLDAvis.gensim

import ast
import  numpy  as np
import pandas as pd

from matplotlib import pyplot as plt
from matplotlib.pyplot import figure
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup
import re


from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split

from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

from sklearn.multiclass import OneVsRestClassifier


import warnings

import string
warnings.filterwarnings('ignore')

plt.style.use('ggplot')


# Unsupervised

### Functions

In [ ]:
# Function to process the text data
def process_text(texts):
    
    bigram = Phrases(texts, min_count=5, threshold=100)
    trigram = Phrases(bigram[texts], min_count=5, threshold=100)

    # Apply the bigram and trigram models
    bigram_mod = Phraser(bigram)
    trigram_mod = Phraser(trigram)
    
    # Apply bigram and trigram models
    texts_P = [trigram_mod[bigram_mod[doc]] for doc in texts]
    
    # Remove stop words
    stop_words = gensim.parsing.preprocessing.STOPWORDS
    texts_F = [[word for word in doc if word not in stop_words] for doc in texts_P]
    
    return texts_F

In [ ]:
def preprocessed_text(text):
    # Define the regular expression pattern for French punctuation marks
    punctuation_pattern = r'[,?!;:…()\[\]«»—/""\'\']'
    
    # Remove HTML tags
    soup = BeautifulSoup(text, "html.parser")
    text_without_tags = soup.get_text()
    
    # Use re.sub() to remove the matched punctuation marks
    cleaned_text = re.sub(punctuation_pattern, '', text_without_tags)
    
    # Tokenization
    tokens = nltk.word_tokenize(cleaned_text.lower())

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens 
              if token not in stop_words]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    # Join the tokens back into a single string
    preprocessed_text = ' '.join(tokens)
    
    return preprocessed_text


### Pre building of the LDA model with bi/trigram  phraser and TF-IDF

In [ ]:
df_LDA = df_non_MLB[['Questions']]

# df_LDA['Questions'] =  df_LDA['Questions'].apply(clean_text)
# Preprocess the text data
df_LDA['text'] = df_LDA['Questions']
df_LDA['clean_text'] = df_LDA['text'].apply(gensim.utils.simple_preprocess, deacc=True)

# Create bigrams and trigrams
text_data = df_LDA['clean_text'].tolist()

bigram = Phrases(text_data, min_count=5, threshold=100)
trigram = Phrases(bigram[text_data], min_count=5, threshold=100)

# Apply the bigram and trigram models
bigram_mod = Phraser(bigram)
trigram_mod = Phraser(trigram)

# Apply bigram and trigram models
texts_P = [trigram_mod[bigram_mod[doc]] for doc in text_data]

# Remove stop words
stop_words = gensim.parsing.preprocessing.STOPWORDS
texts_F = [[word for word in doc if word not in stop_words] for doc in texts_P]

# Create the dictionary
id2word = corpora.Dictionary(texts_F)

# Create the corpus with TF-IDF representation
texts = [id2word.doc2bow(doc) for doc in texts_F]

tfidf = models.TfidfModel(texts)
corpus_tfidf = tfidf[texts]


In [ ]:
# search_word = "c#"
# count = sum(1 for word in processed_data if search_word in word)

# print("Number of occurrences of '",search_word ," ':", count)

In [ ]:
texts_F

### Evaluating  the  coherence score iterating the num-topics parameter  of the  LDA model 

In [ ]:
coherence_scores = []
num_topics_range = range(1, 100, 1)  # Range of num_topics values from 2 to 60 jumping by 2

for num_topics in num_topics_range:
    # Train the LDA model (using LdaMulticore)
    lda_model = gensim.models.LdaMulticore(corpus=corpus_tfidf,
                                           id2word=id2word,
                                           num_topics=num_topics,
                                           random_state=100,
                                           passes=10,
                                           workers=4)

    # Compute coherence score
    coherence_model_lda = CoherenceModel(model=lda_model, texts=texts_F, dictionary=id2word, coherence='c_v')
    coherence_score = coherence_model_lda.get_coherence()
    coherence_scores.append(coherence_score)

    print("num_topics:", num_topics, "Coherence Score:", coherence_score)


# Plot the coherence scores against alpha values
plt.figure(figsize=(15, 9))
# Plot the coherence scores
plt.plot(num_topics_range, coherence_scores, marker='o')
plt.xlabel("num_topics")
plt.ylabel("Coherence Score")
plt.title("Coherence Score vs num_topics")
plt.show()

### Evaluating  the  coherence score iterating the alpha parameter  of the  LDA model 

In [ ]:
# List of alpha values to test
alpha_values = np.arange(0.1, 0.51, 0.01)

# List to store coherence scores for each alpha
coherence_scores = []

# Train LDA models and calculate coherence scores
for alpha in alpha_values:
    lda_model = gensim.models.LdaMulticore(corpus=corpus_tfidf,
                                           id2word=id2word,
                                           alpha=alpha,
                                           num_topics=41,  # Adjust the number of topics as desired
                                           random_state=100,
                                           passes=10,
                                           workers=4)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=texts_F, dictionary=id2word, coherence='c_v')
    coherence_score = coherence_model_lda.get_coherence()
    coherence_scores.append(coherence_score)
    
    print(f"Alpha: {alpha:.2f} - Coherence Score: {coherence_score:.4f}")

# Plot the coherence scores against alpha values
plt.figure(figsize=(15, 9))
plt.plot(alpha_values, coherence_scores, marker='o')
plt.xlabel('Alpha Value')
plt.ylabel('Coherence Score')
plt.title('LDA Model Coherence Score vs. Alpha Value')
plt.grid(True)
plt.show()


### Building the LDA model with optimized parameters (alpha=0.43   , num_topics=41)

In [ ]:
# Train the LDA model (using LdaMulticore)
lda_model = gensim.models.LdaMulticore(corpus=corpus_tfidf,
                                       id2word=id2word,
                                       alpha=0.43,
                                       num_topics=41,  # Adjust the number of topics as desired
                                       random_state=100,
                                       passes=10,
                                       workers=4)

# Compute coherence score
coherence_model_lda = CoherenceModel(model=lda_model, texts=texts_F, dictionary=id2word, coherence='c_v')
coherence_score = coherence_model_lda.get_coherence()

# Print coherence score
print("Coherence Score:", coherence_score)

# Visualize the topics using pyLDAvis
vis_data = pyLDAvis.gensim.prepare(lda_model, corpus_tfidf, id2word)
pyLDAvis.display(vis_data)

### Printout the average tags per questions in our data frame

In [ ]:
# Step 1: Calculate the number of words in each row
num_words_per_row = df_non_MLB['Tags'].apply(len)

# Step 2: Calculate the average number of words
average_words_per_row = num_words_per_row.mean()

print("Average number of tags per question:", round(average_words_per_row))

### Testing the LDA model by suggesting X tags per questions asked

In [ ]:
row_index = 0
number_tag_suggested = 15

input_text = df_LDA['Questions'][row_index]

preprocessed_text_example  =  preprocessed_text(input_text)

clean_input_text = gensim.utils.simple_preprocess(preprocessed_text_example, deacc=True)
processed_input_text = trigram_mod[bigram_mod[clean_input_text]]

input_bow = id2word.doc2bow(processed_input_text)

input_topic_distribution = lda_model.get_document_topics(input_bow, minimum_probability=0.0)
sorted_topics = sorted(input_topic_distribution, key=lambda x: x[1], reverse=True)

most_relevant_topic = sorted_topics[0][0]  # Select the most relevant topic
second_relevant_topic = sorted_topics[1][0]  # Select the second most relevant topic [Ignoring the first one is better on avg]

top_terms = lda_model.show_topic(most_relevant_topic, topn=number_tag_suggested)
suggested_LDA_tags = [term[0] for term in top_terms]

# if suggested_LDA_tags not in ['file', 'string', 'data', 'error', 'class', 'public', 'int', 'app', 'new', 'function', 'test', 'code', 'use', 'user', 'value']:
#     top_terms = lda_model.show_topic(second_relevant_topic, topn=number_tag_suggested)
#     suggested_LDA_tags = [term[0] for term in top_terms]
# else:
#     exit

print("These are the", number_tag_suggested, "suggested tags for your question:", suggested_LDA_tags)
print("Real tags of that question:", df_non_MLB['Tags'][row_index])

# Supervised approach

### FUNCTIONS

In [ ]:
# Getting j_score of the true and predicted values
def j_score(y_true, y_pred):
  jaccard = np.minimum(y_true, y_pred).sum(axis = 1)/np.maximum(y_true, y_pred).sum(axis = 1)
  return jaccard.mean()*100

# Printout the jaccard score of a model
def print_score(y_pred, clf):
  print("Model: ", clf.__class__.__name__)
  print('Jaccard score: {}'.format(j_score(y_test, y_pred)))
  print('----')
  

### Data preparation

In [ ]:
df_MLB.head()

In [ ]:
df_OG_TT['Title']

In [ ]:

X = df_MLB[['Questions']]
y = df_MLB.drop('Questions', axis=1)
titleT = df_OG_TT['Title']

In [ ]:
df_MLB.columns

In [ ]:
tfidf_test = TfidfVectorizer(analyzer='word', stop_words='english')
tfidf_test.fit(X['Questions'])
X_TFIDF_test = tfidf_test.transform(X['Questions'])

X_TFIDF_test.shape , y.shape

In [ ]:
X_train_test , X_test_test , y_train  , y_test =  train_test_split (X_TFIDF_test , y , test_size = 0.2 , random_state=30 )
X_train_test .shape  , X_test_test .shape  , y_train.shape   , y_test.shape 

### Choosing a first classifier between SGD , LR and SVC

In [ ]:
sgd = SGDClassifier()
lr = LogisticRegression(solver='lbfgs')
svc = LinearSVC()

In [ ]:
for classifier in [sgd, lr, svc]:
  clf = OneVsRestClassifier(classifier)
  clf.fit(X_train_test , y_train)
  y_pred = clf.predict(X_test_test)
  print_score(y_pred, classifier)
  #52.63936479864412

## TF IDF optimization

### I) Searching for ngram_range optimal value with TFIDF fit on title and entire questions for L1 and L2 SVC models 

#### i) 1- L2 step on the entire question

In [ ]:
ngram_ranges = [(1, 1), (1, 2), (1, 3) , (2,2)]  # Different ngram_range values to iterate over

for ngram_range in ngram_ranges:
    tfidf = TfidfVectorizer(analyzer='word', stop_words='english', ngram_range=ngram_range)
    tfidf.fit(X['Questions'])
    
    X_TFIDF = tfidf.transform(X['Questions'])
    
    X_train, X_test, y_train, y_test = train_test_split(X_TFIDF, y, test_size=0.2, random_state=30)
    
    print(f"Processing ngram_range: {ngram_range}")
    
    for classifier in [LinearSVC(penalty='l2', dual=True)]:
        clf = OneVsRestClassifier(classifier)
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        print_score(y_pred, classifier)  # You need to define the print_score function
    

#### i) 2- L2 step on the title only

In [ ]:
ngram_ranges = [(1, 1), (1, 2), (1, 3) , (2,2)]  # Different ngram_range values to iterate over

for ngram_range in ngram_ranges:
    tfidf = TfidfVectorizer(analyzer='word', stop_words='english', ngram_range=ngram_range)
    tfidf.fit(titleT)
    
    X_TFIDF = tfidf.transform(X['Questions'])
    
    X_train, X_test, y_train, y_test = train_test_split(X_TFIDF, y, test_size=0.2, random_state=30)
    
    print(f"Processing ngram_range: {ngram_range}")
    
    for classifier in [LinearSVC(penalty='l2', dual=True)]:
        clf = OneVsRestClassifier(classifier)
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        print_score(y_pred, classifier)  # You need to define the print_score function
    

#### ii) 1- L1 step on the entire question

In [ ]:
ngram_ranges = [(1, 1), (1, 2), (1, 3) , (2,2)]  # Different ngram_range values to iterate over

for ngram_range in ngram_ranges:
    tfidf = TfidfVectorizer(analyzer='word', stop_words='english', ngram_range=ngram_range)
    tfidf.fit(X['Questions'])
    
    X_TFIDF = tfidf.transform(X['Questions'])
    
    X_train, X_test, y_train, y_test = train_test_split(X_TFIDF, y, test_size=0.2, random_state=30)
    
    print(f"Processing ngram_range: {ngram_range}")
    
    for classifier in [LinearSVC(penalty='l1', dual=False)]:
        clf = OneVsRestClassifier(classifier)
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        print_score(y_pred, classifier)  # You need to define the print_score function
    

#### ii) 2- L1 step on the title only

In [ ]:
ngram_ranges = [(1, 1), (1, 2), (1, 3) , (2,2)]  # Different ngram_range values to iterate over

for ngram_range in ngram_ranges:
    tfidf = TfidfVectorizer(analyzer='word', stop_words='english', ngram_range=ngram_range)
    tfidf.fit(titleT)
    
    X_TFIDF = tfidf.transform(X['Questions'])
    
    X_train, X_test, y_train, y_test = train_test_split(X_TFIDF, y, test_size=0.2, random_state=30)
    
    print(f"Processing ngram_range: {ngram_range}")
    
    for classifier in [LinearSVC(penalty='l1', dual=False)]:
        clf = OneVsRestClassifier(classifier)
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        print_score(y_pred, classifier)  # You need to define the print_score function
    

## Linear SVC model optimization

### II) Searching for C optimal value for SVC models

#### i) TF IDF optimized for L1 SVC model

In [ ]:
tfidf = TfidfVectorizer(analyzer='word', stop_words='english' , ngram_range=(1,2))
tfidf.fit(titleT)

X_TFIDF = tfidf.transform(X['Questions'])

X_train , X_test , y_train  , y_test =  train_test_split (X_TFIDF , y , test_size = 0.2 , random_state=30 )

X_train.shape  , X_test.shape  , y_train.shape   , y_test.shape 

#### Evaluating J score by iterating C paramter of SVC with L1 parameter (TFIDF  ngram_range=(1,2))

In [ ]:
C_values = np.arange(1, 10.5, 0.5)  # Create an array of C values from 1 to 10 with a step of 0.5
jaccard_scores = []

for C in C_values:
    classifier = LinearSVC(C=C , penalty = 'l1' , dual=False)
    clf = OneVsRestClassifier(classifier)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print("For ", C , "C  parameter value you  have these results")
    print_score(y_pred, classifier)
    jaccard_score = j_score(y_test, y_pred)  # Replace with the actual Jaccard score calculation
    jaccard_scores.append(jaccard_score)

# Plot the Jaccard scores
plt.figure(figsize=(15, 9))
plt.plot(C_values, jaccard_scores , marker='o')
plt.xlabel('C Parameter')
plt.ylabel('Jaccard Score')
plt.title('Jaccard Score vs C Parameter WITH L1')
plt.show()

#### ii) TF IDF optimized for L2 SVC model

In [ ]:
tfidf = TfidfVectorizer(analyzer='word', stop_words='english' , ngram_range=(1,1))
tfidf.fit(titleT)

X_TFIDF = tfidf.transform(X['Questions'])

X_train , X_test , y_train  , y_test =  train_test_split (X_TFIDF , y , test_size = 0.2 , random_state=30 )

X_train.shape  , X_test.shape  , y_train.shape   , y_test.shape 

#### Evaluating J score by iterating C paramter of SVC with L2 parameter

In [ ]:
C_values = np.arange(1, 10.5, 0.5)  # Create an array of C values from 1 to 10 with a step of 0.5
jaccard_scores = []

for C in C_values:
    classifier = LinearSVC(C=C ,  penalty = 'l2' , dual=True)
    clf = OneVsRestClassifier(classifier)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print("For ", C , "C  parameter value you  have these results")
    print_score(y_pred, classifier)
    jaccard_score = j_score(y_test, y_pred)  # Replace with the actual Jaccard score calculation
    jaccard_scores.append(jaccard_score)

# Plot the Jaccard scores
plt.figure(figsize=(15, 9))
plt.plot(C_values, jaccard_scores , marker='o')
plt.xlabel('C Parameter')
plt.ylabel('Jaccard Score')
plt.title('Jaccard Score vs C Parameter WITH L2')
plt.show()

### III) Final model  : TFIDF ngram_range=(1,2) , SVC (C=1.5 with L1) is the best option

In [ ]:
tfidf = TfidfVectorizer(analyzer='word', stop_words="english", ngram_range=(1, 2))
tfidf.fit(titleT)
X_TFIDF = tfidf.transform(X['Questions'])

X_train , X_test , y_train  , y_test =  train_test_split (X_TFIDF , y , test_size = 0.2 , random_state=30 )

X_train.shape  , X_test.shape  , y_train.shape   , y_test.shape 

In [ ]:
for classifier in [LinearSVC(C=1.5 , penalty = 'l1' , dual=False) ]:
  clf = OneVsRestClassifier(classifier)
  clf.fit(X_train, y_train)
  y_pred = clf.predict(X_test)
  print_score(y_pred, classifier)
  # 55,13

### IV) Testing the model 

In [ ]:
input_test = 'How can I fine tune a BERT model ?'
input_test_1 = 'How can I input an image into my website ?'
input_test_2 = 'How to add interactivity with to my web site in a declarative way on the front end?'
input_test_3 = 'What is the difference between high level and low level language programming ?'
input_test_4 ='How to creat a boss script using c# in a video game ?'

input_test_fail = 'What is the difference between supervised and unsupervised learning in the machine learning field, and what are their respective utilities?'

test_text = input_test_2  # Take the first element of the list as a single string

preprocessed_test = preprocessed_text(test_text)

input_test_T = tfidf.transform([preprocessed_test])  # Pass the preprocessed text as a list

predicted_df = pd.DataFrame(clf.predict(input_test_T), columns=y.columns)

# Filter columns where there's only one 1 and print their names
columns_with_single_1 = predicted_df.columns[predicted_df.apply(lambda col: col.sum() == 1)].tolist()

print("The question is : " , test_text)
print("Here are the predicted tags:")
print(columns_with_single_1)


# Deep Learning (BERT fine-tunning) using PYTORCH

In [ ]:
import warnings
warnings.filterwarnings("ignore", message="numpy.*")
import os

from torch.optim.lr_scheduler import StepLR
import torch.optim as optim

import string
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, AutoModelForSequenceClassification
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torch
from tqdm import tqdm
from transformers import get_linear_schedule_with_warmup

import multiprocessing

import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau , CosineAnnealingLR
from sklearn.metrics import f1_score, jaccard_score
import ast
import numpy as np
from scipy import stats
import math
from sklearn import datasets
from sklearn import metrics
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import copy

import random

plt.style.use('ggplot')


# FUNCTIONS

### Model  evaluation function (with freezing all  layers as  an option)

In [ ]:
def model_evaluation(model, data_loader, loss_fn, freeze_all=True):

    for param in model.parameters():
        param.requires_grad = freeze_all

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    model.eval()

    total_correct_test = 0
    total_samples_test = 0
    total_loss = 0.0

    predictions = []
    targets_list = []

    with torch.no_grad(), tqdm(total=len(data_loader), desc="Testing") as pbar:
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            targets = batch['targets'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)

            loss = loss_fn(outputs.logits, targets)
            total_loss += loss.item()

            predicted_probs = torch.sigmoid(outputs.logits)
            predicted_tags = (predicted_probs >= 0.5).float()

            total_correct_test += torch.sum(torch.eq(predicted_tags, targets)).item()
            total_samples_test += len(targets)

            predictions.append(predicted_tags.cpu().numpy())
            targets_list.append(targets.cpu().numpy())

            pbar.update(1)

    avg_test_accuracy = total_correct_test / total_samples_test
    avg_loss = total_loss / len(data_loader)

    predictions = np.concatenate(predictions)
    targets_list = np.concatenate(targets_list)

    avg_f1_micro = f1_score(targets_list, predictions, average='micro')
    avg_jaccard_micro = jaccard_score(targets_list, predictions, average='micro')

    avg_f1_macro = f1_score(targets_list, predictions, average='macro')
    avg_jaccard_macro = jaccard_score(targets_list, predictions, average='macro')
  
    print("------------------------------------- Evaluation DONE ! --------------------------------------")
    print(f"Test Accuracy: {avg_test_accuracy:.3f}")
    print(f"Average Loss: {avg_loss:.5f}")
    print(f"Micro-average F1 Score: {avg_f1_micro:.4f} - Macro-average F1 Score: {avg_f1_macro:.4f}")
    print(f"Micro-average Jaccard Score: {avg_jaccard_micro:.4f}  - Macro-average Jaccard Score: {avg_jaccard_macro:.4f}")


### Best learning rate finder (with elbow method)

In [ ]:
def lr_best_finder(model, data_loader, loss_fn, lr_values, sample_size = 0.1 , lr_finder=False ):

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Create a deep copy of the model
    edited_model = copy.deepcopy(model)

    edited_model.train()
    output_layer_index = -1  # Layers index's not  to freeze (-1  means  the  last one)

    for idx, param in enumerate(edited_model.parameters()):  # Freeze all layers except the output_layer_index (last one)
        if idx < output_layer_index:
            param.requires_grad = False
        else:
            param.requires_grad = True

    edited_model.to(device)
    loss_fn.to(device)

    batch = next(iter(data_loader))
    losses = []

    for lr in lr_values:
        optimizer = optim.AdamW(edited_model.parameters(), lr=lr)
        total_loss = 0.0
        num_batches = int(len(data_loader) * sample_size)

        for iteration in tqdm(range(num_batches), desc=f"Training for LR {lr:.2e}"):
            
            # Create a random subset of the data for each iteration to guarantee a good generalization with shuffling and randomness
            shuffled_indices = list(range(len(data_loader.dataset)))
            random.shuffle(shuffled_indices)
            subset_indices = shuffled_indices[:data_loader.batch_size]
            subset = Subset(data_loader.dataset, subset_indices)

            # Create a batch sampler to iterate over the subset in batches
            batch_sampler = BatchSampler(SequentialSampler(subset), batch_size=data_loader.batch_size, drop_last=False)

            for batch_indices in batch_sampler:
                batch = [subset[i] for i in batch_indices]

                batch_inputs = {
                    'input_ids': torch.stack([sample['input_ids'] for sample in batch]).to(device),
                    'attention_mask': torch.stack([sample['attention_mask'] for sample in batch]).to(device),
                }
                batch_labels = torch.stack([sample['targets'] for sample in batch]).to(device)

                optimizer.zero_grad()

                # Forward pass
                outputs = edited_model(**batch_inputs)
                logits = outputs.logits

                # Calculate loss
                loss = loss_fn(logits, batch_labels)
                total_loss += loss.item()

                # Backpropagation to update the parameter of the chosen model
                loss.backward()
                optimizer.step()

        average_loss = total_loss / num_batches
        losses.append(average_loss)

    # Plot size
    plt.figure(figsize=(15, 9))

    # plotting a large zoomed out graph and outputing the lowest loss point
    if lr_finder == False :
      plt.plot(lr_values, losses)
      plt.xscale('log')
      plt.xlabel('Learning Rate')
      plt.ylabel('Loss')
      plt.title('Learning Rate Range Test')

      best_lr_index = losses.index(min(losses))
      lowest_lr = lr_values[best_lr_index]

      plt.scatter(lowest_lr, min(losses), color='red', marker='o', label=f'Lowest Loss : {lowest_lr:.0e}')

      plt.legend()  # Add a legend to label the red dot
      plt.show()

      print(f"Learning rate value of the lowest loss value point is: {lowest_lr:.0e}")
      
      return lowest_lr
      
    # plotting a zoomed in graph and  outputing the elbow point as the best option  
    else :
      # Calculate differences between consecutive losses
      loss_diffs = [losses[i] - losses[i - 1] for i in range(1, len(losses))]
      # Calculate second differences between consecutive loss differences
      loss_diff_diffs = np.diff(loss_diffs)

      # Find the index of the point of inflection (elbow)
      inflection_index = np.argmax(loss_diff_diffs) + 1

      # Choose the learning rate associated with the point of inflection
      best_lr = lr_values[inflection_index]

      # Plot the results
      plt.plot(lr_values, losses)
      plt.scatter(best_lr, losses[inflection_index], color='green', label=f'Best LR: {best_lr:.0e}')
      plt.xscale('log')
      plt.xlabel('Learning Rate')
      plt.ylabel('Loss')
      plt.title('Learning Rate Range Test')
      plt.legend()
      plt.show()

      print(f"Learning rate value at the point of inflection (elbow) is: {best_lr:.0e}")

      return best_lr


### Fine tunning function

In [ ]:
def fine_tune_bert(model, train_loader, loss_fn, optimizer, num_epochs=5, scheduler=None):

    # devine to set everything either through gpu calculation or cpu,but atleast synchronized
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model.to(device)
    loss_fn.to(device)
    
    output_layer_index = -1  
    
    model.train()
    
    # Freezing all layers expect the last one (I have tested unfreezing as you go method,or unfreezing all layers and  2 other method the best one is freezing all of them expect thel ast one)
    for idx, param in enumerate(model.parameters()): 
        if idx < output_layer_index:
            param.requires_grad = False
        else:
            param.requires_grad = True
    
    # Going through the  epochs with a beautiful tqdm bar progression
    for epoch in range(num_epochs):
        train_loss = 0
        total_correct = 0
        total_samples = 0
        all_predicted_tags = []
        all_targets = []
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False)
        
        # Going through  the  dataset using batchs (size = 16 in  this case),and  updating the  parameters of the  model  after each batch by a forward  pass,loss calculation and a backpropagation process and I added a  schedule option if needed (Cosine  , ReduceLROnplateau or decayingLR  ect . .. )
        for batch in progress_bar:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            targets = batch['targets'].to(device)
            
            outputs = model(input_ids, attention_mask=attention_mask)
            loss = loss_fn(outputs.logits, targets)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            
            predicted_probs = torch.sigmoid(outputs.logits) # Sigmoid because  it is a  multi label text classification,and softmax if it is a  mutli class task
            predicted_tags = (predicted_probs >= 0.5).float()
            
            total_correct += torch.sum(torch.eq(predicted_tags, targets)).item()
            total_samples += len(targets)
            
            all_predicted_tags.extend(predicted_tags.cpu())
            all_targets.extend(targets.cpu())
            
            progress_bar.set_postfix({
                'Loss': loss.item(),
            })
            
        # Registering some evaluiation metrics 
        all_predicted_tags = torch.stack(all_predicted_tags)
        all_targets = torch.stack(all_targets)
        
        predicted_np = all_predicted_tags.numpy()
        targets_np = all_targets.numpy()
        
        f1_micro = f1_score(targets_np, predicted_np, average='micro')
        f1_macro = f1_score(targets_np, predicted_np, average='macro')
        
        jaccard_micro = jaccard_score(targets_np, predicted_np, average='micro')
        jaccard_macro = jaccard_score(targets_np, predicted_np, average='macro')
        
        avg_accuracy = total_correct / total_samples
        
        print(f"Epoch {epoch+1}/{num_epochs} - Loss: {(train_loss/len(train_loader)):.5f} - Accuracy: {avg_accuracy:.3f} - F1 Micro: {f1_micro:.4f} - F1 Macro: {f1_macro:.4f} - Jaccard Micro: {jaccard_micro:.4f} - Jaccard Macro: {jaccard_macro:.4f}" )
        
        #Schedule option 
        if scheduler is not None:
            scheduler.step()

### Suggesting tags

In [ ]:
def predict_tags(model, input_question, threshold=0.5):
    input_question.lower()

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    
    input_ids = tokenizer.encode(input_question, max_length=tokenizer.model_max_length, truncation=True)
    input_ids = torch.tensor(input_ids).unsqueeze(0).to(device)
    
    with torch.no_grad():
        outputs = model(input_ids)
    
    predicted_probabilities = torch.nn.functional.sigmoid(outputs.logits)   # Sigmoid because  it is a  multilabel text classification,and softmax if it is a  mutli class task
    tag_probabilities = predicted_probabilities.squeeze().tolist()
    
    total_probability = round(sum(tag_probabilities) , 6)

    tag_labels = y.columns  

    decimal_places = 3
    
    tag_predictions = [{"tag": tag_labels[i], "probability": prob} for i, prob in enumerate(tag_probabilities) if prob > threshold]
    
    sorted_tag_predictions = sorted(tag_predictions, key=lambda x: x["probability"], reverse=True)

    print(f"Here is the sum of the total probability of all 60 tags :  {total_probability} " )

    # Print the sorted tag predictions above the threshold
    for tag_prediction in sorted_tag_predictions:
        rounded_probability = round(tag_prediction['probability'], decimal_places)
        print(f"Tag: {tag_prediction['tag']}, Probability: {rounded_probability}")
    
    #returning the suggested tags in an array format
    return [item['tag'] for item in sorted_tag_predictions]

### Data structure of my dataset

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, questions, targets, tokenizer, max_length):
        self.questions = questions
        self.targets = targets
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, item):
        question = str(self.questions[item])
        target = self.targets[item]

        encoding = self.tokenizer.encode_plus(
            question,
            add_special_tokens=True,
            max_length=self.max_length,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'targets': torch.tensor(target, dtype=torch.float),
        }

### SPLITING DATA INTO TRAIN/TEST INPUT/OUTPUT

In [ ]:

# Split data into train and test sets
train_df, test_df = train_test_split(df_MLB, test_size=0.2, random_state=42)

# Extract questions and corresponding tags
train_questions = train_df['Questions'].tolist()
train_tags = train_df.drop(columns=['Questions']).values.tolist()

test_questions = test_df['Questions'].tolist()
test_tags = test_df.drop(columns=['Questions']).values.tolist()

y = df_MLB.drop('Questions' , axis=1)
all_tags = y.columns
labels_number = all_tags.shape[0]

### SEARCHING FOR THE BEST MAX_LENGTH VALUE 

In [ ]:
# Calculate the number of words for each sentence
sentence_lengths = df_MLB['Questions'].apply(lambda x: len(x.split()))

# Create a histogram
plt.figure(figsize=(10, 6))
plt.hist(sentence_lengths, bins=50, color='blue', alpha=0.7)
plt.xlabel('Number of Words')
plt.ylabel('Frequency')
plt.title('Distribution of Sentence Lengths')
plt.grid(True)
plt.show()

In [ ]:
# Calculate the average number of words
average_num_words = df_MLB['Questions'].apply(lambda x: len(x.split())).mean()

# Calculate the maximum number of words
max_num_words = df_MLB['Questions'].apply(lambda x: len(x.split())).max()

# Calculate the minimum number of words
min_num_words = df_MLB['Questions'].apply(lambda x: len(x.split())).min()

# Safety margin (1/3 for this case)
safety_margin = 0.33*average_num_words

# Calculate max_length based on average_num_words + safety_margin
max_length = int(average_num_words + safety_margin)

print(f"Average number of words per sentence: {average_num_words:.2f}")
print(f"Maximum number of words per sentence: {max_num_words}")
print(f"Minimum number of words per sentence: {min_num_words}")
print(f"Setting max_length to: {max_length}")


### Data loading / preparation

In [ ]:
# Set maximum sequence length
max_seq_length = 256
batch_size_number  =  16

model_name = 'bert-base-uncased'

# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained(model_name)

# Create DataLoader for training and testing data
train_dataset = CustomDataset(train_questions, train_tags, tokenizer, max_seq_length)
train_loader = DataLoader(train_dataset, batch_size=batch_size_number, shuffle=True)

test_dataset = CustomDataset(test_questions, test_tags, tokenizer, max_seq_length)
test_loader = DataLoader(test_dataset, batch_size=batch_size_number, shuffle=False)

# Load the pre-trained BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=labels_number)

# Define the optimizer
optimizer = optim.AdamW(model.parameters())

loss_fn = nn.BCEWithLogitsLoss()


# EVALUATING THE BASE BERT MODEL DIRECTLY WITHOUT FINE TUNNING 

In [ ]:
model_evaluation(model, train_loader, loss_fn)

# FINDING  THE  BEST LEARNING RATE VALUE

In [ ]:
# Finding the first range 
lr_values  = [1e-7 , 1e-6  , 1e-5 , 1e-4 , 1e-3 ,  1e-2  , 1e-1]

lowest_loss_lr = lr_best_finder(model , train_loader , loss_fn , lr_values)

In [ ]:
# Finding the real precise value
lr_values = [1e-5 , 2e-5, 3e-5 , 4e-5 , 5e-5 , 6e-5 , 7e-5  , 8e-5 , 9e-5 , 1e-4 , 2e-4 , 3e-4]

best_lr_value = lr_best_finder(model , train_loader , loss_fn , lr_values , lr_finder = True)

# FINE TUNNING PROCESS

In [ ]:
# Fine-tuning the BERT model
num_epochs = 6

# Define the optimizer
optimizer = optim.AdamW(model.parameters(), lr=best_lr_value)

# Define the learning rate scheduler
scheduler_cosine = CosineAnnealingLR(optimizer, T_max=num_epochs)

fine_tune_bert(model, train_loader, loss_fn, optimizer, num_epochs=num_epochs, scheduler=scheduler_cosine)

## SAVING THE FINE TUNNED MODEL (default : Google Collab)

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

model_path = "/content/drive/My Drive/bert_model_2e-5.pth"  # This path  is for  google collab user,change it at your will
torch.save(model.state_dict(), model_path)

# ---------------------------------------------------------------------------------------------
# This below is the code to load the model localy
# ---------------------------------------------------------------------------------------------

# # Get the current working directory (directory of the Jupyter Notebook)
# notebook_directory = %pwd

# model_filename = "bert_model_2e-5.pth"
# model_path = os.path.join(notebook_directory, model_filename)

# # Assuming you have a 'model' variable containing your model
# torch.save(model.state_dict(), model_path)

## LOADING THE FINE TUNNED MODEL (default : Google Collab)

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

loaded_model = BertForSequenceClassification.from_pretrained(
    model_name,
    num_labels=labels_number
)

# Load the saved model state dictionary
model_path = "/content/drive/My Drive/bert_model_2e-5.pth"  

loaded_model.load_state_dict(torch.load(model_path))

# ---------------------------------------------------------------------------------------------
# This below is the code to load the model localy
# ---------------------------------------------------------------------------------------------

# # Create the model instance
# loaded_model = BertForSequenceClassification.from_pretrained(
#     model_name,
#     num_labels=labels_number
# )

# # Get the current working directory (directory of the Jupyter Notebook)
# notebook_directory = %pwd

# # Load the saved model state dictionary
# model_filename = "bert_model_2e-5.pth"
# model_path = os.path.join(notebook_directory, model_filename)

# loaded_model.load_state_dict(torch.load(model_path)) # NOTE : The model has been fine tunned using T4 GPU,loading it with a CPU will not work !

## EVALUATING THE FINE TUNNED MODEL

In [ ]:
model_evaluation(loaded_model, test_loader, loss_fn)

## PREDICTING TAGS WITH AN INPUT QUESTION 

In [ ]:
index_row =  4234

questions1 = df_MLB['Questions'][index_row]
input_test = 'How can I fine tune a BERT model ?'
input_test_1 = 'How can I input an image into my website ?'
input_test_2 = 'How to add interactivity with to my web site in a declarative way on the front end?'
input_test_3 = 'What is the difference between high level and low level language programming ?'
input_test_4 = 'How to creat a boss script using c# in a video game ?'



input_question = input_test_3

preprocessed_question = preprocessed_text(input_question)

suggested_tags_BERT = predict_tags(loaded_model, input_question, threshold=0.1)

print("------------------------------------------------------------------------")
print("Real tags :", df_OG_TT['Tags'][index_row])
